# DFT: The LDA kernel
## I. Theory

Previously we described the DFT Fock matrix as
$$F^{DFT}_{\mu\nu} = H_{\mu\nu} + 2J[D]_{\mu\nu} - \zeta K[D]_{\mu\nu} + V^{\rm{xc}}_{\mu\nu}$$
upon examination it is revealed that the only quantity that we cannot yet compute is $V^{\rm{xc}}$. 

Here we will explore the local density approximation (LDA) functionals where $V^{\rm{xc}} = f[\rho(\hat{r})]$. For these functionals the only required bit of information is the density at the grid point. As we discussed the grid last chapter we will now focus exactly on how to obtain the density on the grid.

Before we begin we should first recall that the Fock matrix is the derivative of the energy with respect to atomic orbitals. Therefore, the $V^{\rm{xc}}$ matrix is not the XC energy, but the derivate of that energy, which can expressed as $\frac{\partial e_{\rm{xc}}}{\partial\rho}$. 

In [1]:
using PyCall: pyimport
psi4 = pyimport("psi4")
np   = pyimport("numpy") # used only to cast to Julia Arrays

mol = psi4.geometry("""
He
symmetry c1
""")
psi4.set_options(Dict("BASIS" => "cc-pvdz",
                      "DFT_SPHERICAL_POINTS" => 6,
                      "DFT_RADIAL_POINTS" => 5))

psi4.core.set_output_file("output.dat", false)

svwn_w, wfn = psi4.energy("SVWN", return_wfn=true)
Vpot = wfn.V_potential()

PyObject <psi4.core.VBase object at 0x14a0ef6b0>

## 2. Density on a Grid
The density on the grid can be expressed as
$$\rho(\hat{r}) = \sum\limits_{\mu\nu} D_{\mu\nu}\;\phi_\mu(\hat{r})\phi_\nu(\hat{r})$$

Recall that we compute DFT quanties on a grid, so $\hat{r}$ will run over a grid instead of all space. Using this we can build collocation matrices that map between atomic orbital and grid space $$\phi_\mu(\hat{r}) \rightarrow \phi_\mu^p$$
where our $p$ index will be the index of individual grid points. Our full expression becomes:

$$\rho_p = \phi_\mu^p D_{\mu\nu} \phi_\nu^p$$

To compute these quantities let us first remember that the DFT grid is blocked loosely over atoms. It should now be apparent to why we do this, consider the $\phi_\mu^p$ objects. The total size of this object would be `nbf` $\times$ `npoints`. To put this in perspective a moderate size molecule could have 1e4 basis functions and 1e8 grid points, so about 8 terabytes of data! As this object is very sparse it is much more convenient to store the grid and compute $\phi\mu^p$ matrices on the fly. 

We then need object to compute $\phi_\mu^p$. 

In [2]:
# Grab a "points function" to compute the Phi matrices
points_func = Vpot.properties()[1]

# Grab a block and obtain its local mapping
block = Vpot.get_block(1)
npoints = block.npoints()
lpos = np.array(block.functions_local_to_global()) .+ 1
println("Local basis function mapping")
display(lpos)

# Copmute phi, note the number of points and function per phi changes.
phi = np.array(points_func.basis_values()["PHI"])[1:npoints, 1:size(lpos)[1]]
println("\nPhi Matrix")
display(phi)

5-element Array{Int64,1}:
 1
 2
 3
 4
 5

Local basis function mapping


25×5 Array{Float64,2}:
 1.22732e-18  1.59271e-5   0.0          6.46773e-18   0.0
 1.22732e-18  1.59271e-5   0.0          0.0           6.46773e-18
 1.22732e-18  1.59271e-5   0.0          0.0          -6.46773e-18
 1.22732e-18  1.59271e-5   6.46773e-18  0.0           0.0
 1.22732e-18  1.59271e-5  -6.46773e-18  0.0           0.0
 0.000171235  0.0395228    0.0          0.0010179     0.0
 0.000171235  0.0395228    0.0          0.0           0.0010179
 0.000171235  0.0395228    0.0          0.0          -0.0010179
 0.000171235  0.0395228    0.0010179    0.0           0.0
 0.000171235  0.0395228   -0.0010179    0.0           0.0
 0.188418     0.211723     0.0          0.529558      0.0
 0.188418     0.211723     0.0          0.0           0.529558
 0.188418     0.211723     0.0          0.0          -0.529558
 0.188418     0.211723     0.529558     0.0           0.0
 0.188418     0.211723    -0.529558     0.0           0.0
 1.07233      0.280679     0.0          0.485241      0.0
 1.07233   


Phi Matrix


## 3. Evaluating the kernel

After building the density on the grid we can then compute the exchange-correlation $f_{xc}$ at every gridpoint. This then need to be reintegrated back to atomic orbital space which can be accomplished like so:

$$V^{\rm{xc}}_{pq}[D_{pq}] = \phi_\mu^a\;\phi_\nu^a\;\; w^a\;f^a_{\rm{xc}}{(\phi_\mu^p D_{\mu\nu} \phi_\nu^p)}$$

Where $w^a$ is our combined Truetler and Lebedev weight at every point.

Unlike SCF theory where the SCF energy can be computed as the sum of the Fock and Density matrices the energy for XC kernels must be computed in grid space. Fortunately, the energy is simply defined as:

$$e_{\rm{xc}} = w^a f^a_{\rm{xc}}$$

We can now put all the pieces together to compute $e_{\rm{xc}}$ and $\frac{\partial E_{\rm{xc}}}{\partial\rho}= V^{\rm{xc}}$.

In [4]:
D = np.array(wfn.Da())

V = zero(D)

rho = []
points_func = Vpot.properties()[1]
superfunc = Vpot.functional()

xc_e, V = let xc_e = 0.0, rho = rho
   # Loop over the blocks
   for b in 1:Vpot.nblocks()
       
       # Obtain block information
       block = Vpot.get_block(b-1)
       points_func.compute_points(block)
       npoints = block.npoints()
       lpos = np.array(block.functions_local_to_global()) .+ 1
       
       # Obtain the grid weight
       w = np.array(block.w())

       # Compute ϕᴾμ!
       phi = np.array(points_func.basis_values()["PHI"])[1:npoints, 1:size(lpos)[1]]
       
       # Build a local slice of D
       lD = D[lpos,lpos]
       
       # Copmute ρ
       # ρᴾ = ϕᴾμ Dμν ϕᴾν
       rho = 2vec(sum( (phi * lD) .* phi, dims=2))

       inp = Dict()
       inp["RHO_A"] = psi4.core.Vector.from_array(rho)
       
       # Compute the kernel
       ret = superfunc.compute_functional(inp, -1)
       
       # Compute the XC energy
       vk = np.array(ret["V"])[1:npoints]
       xc_e = w .* vk
           
       # Compute the XC derivative.
       v_rho_a = np.array(ret["V_RHO_A"])[1:npoints]
       # Vab = ϕᴾ_b Vᴾ Wᴾ ϕᴾ_a
       Vtmp = phi' * (v_rho_a .* w .* phi)

       # Add the temporary back to the larger array by indexing, ensure it is symmetric
       V[lpos, lpos] += 0.5(Vtmp + Vtmp')
   end
   xc_e, V
end


println("XC Energy: ")
display(xc_e)
println("V matrix:")
display(V)

println("\nMatches Psi4 V: ", np.allclose(V, wfn.Va()))

25-element Array{Float64,1}:
 -6.519167903047856e-11
 -6.519167903047856e-11
 -6.519167903047856e-11
 -6.519167903047856e-11
 -6.519167903047856e-11
 -0.003017677991539832
 -0.003017677991539832
 -0.003017677991539832
 -0.003017677991539832
 -0.003017677991539832
 -0.09338029347594821
 -0.09338029347594819
 -0.09338029347594819
 -0.09338029347594823
 -0.09338029347594817
 -0.07147329076318813
 -0.07147329076318813
 -0.07147329076318813
 -0.07147329076318813
 -0.07147329076318813
 -0.0005738633785625028
 -0.0005738633785625028
 -0.0005738633785625028
 -0.0005738633785625028
 -0.0005738633785625028

XC Energy: 


5×5 Array{Float64,2}:
 -0.826222     -0.443305     -2.80973e-17   0.0       -8.30684e-18
 -0.443305     -0.410914     -2.96189e-17   0.0       -8.07065e-18
 -2.80973e-17  -2.96189e-17  -0.732703      0.0        0.0
  0.0           0.0           0.0          -0.732703   0.0
 -8.30684e-18  -8.07065e-18   0.0           0.0       -0.732703

V matrix:

Matches Psi4 V: true


Refs:
- Johnson, B. G.; Fisch M. J.; *J. Chem. Phys.*, **1994**, *100*, 7429